In [3]:
import json
import sys

sys.path.append('..')
from tools.TextProcessing import build_word_tree_v2, process_keywords, clean_text
from tools.BasicUtils import my_write, my_csv_read, MultiThreading, get_wikipedia_entity

<h2> Select keyword file and generate keyword list

In [5]:
# Collect keywords from terms-cs-cfl-epoch200.txt
stable_kw = []
unstable_kw = []
r = my_csv_read('../data/raw_data/terms-cs-cfl-epoch200.txt', delimiter='\t')
candidate_kw_list = [clean_text(item[0]) for item in r if float(item[1]) > 0.1 and item[2] == '1']
stable_kw, unstable_kw = process_keywords(candidate_kw_list)
# Save keywords
my_write('../data/corpus/keyword.txt', stable_kw)
# Generate word tree (25 seconds)
build_word_tree_v2('../data/corpus/keyword.txt', '../data/corpus/wordtree.pickle', '../data/corpus/token.txt')

transform keywords into index


100%|██████████| 17028/17028 [00:00<00:00, 874373.90it/s]


start building wordtree


100%|██████████| 17028/17028 [00:00<00:00, 72473.50it/s]

Building word tree is accomplished with 17028 words added
Total time taken in :  build_word_tree_v2 0.2862679958343506


<h2> Bind word with wikipedia page (be out-dated)

In [ ]:
# Collect keywords that can be linked to a wikipedia page (11 minutes)
mt = MultiThreading()
ret = mt.run(lambda x: (x, get_wikipedia_entity(x)), stable_kw, thread_num=10)
kw2ent_dict = {item[0]:item[1] for item in ret if item[1] is not None}
json.dump(kw2ent_dict, open('../data/corpus/kw_ent_map.json', 'w'))
final_keywords = list(kw2ent_dict.keys())

In [5]:
# Store the useful wikipedia entities
wiki_ents = list(set(kw2ent_dict.values()))
my_write('../data/corpus/wikipedia_ent.txt', wiki_ents)

<h2> Store the keyword list and build word tree (be out-dated)

In [ ]:
# Save keywords
my_write('../data/corpus/keyword_f.txt', final_keywords)
# Generate word tree (25 seconds)
build_word_tree('../data/corpus/keyword_f.txt', '../data/corpus/wordtree.json', '../data/corpus/entity.txt')